In [1]:
from FNN import FNN
from Layer import Layer
from Neuron import Neuron
import numpy as np
from ActivFunctions import  *
from InitFunctions import  *
from SuppFunctions import  *
from ErrorClasses import *

Neuron Class functionality testing

In [2]:
#zero initialization
neuron = Neuron(2,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [3]:
#random initialization
neuron = Neuron(2,identity,method_ini = "RandomNor", random_mean = -1000.0, random_std = 10.0)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [4]:
#initialization from list
vector = [1.0,2.0,3.5]
neuron = Neuron(vector,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [5]:
#initialization from vector
vector = np.array([1.0,2.0,3.5])
neuron = Neuron(vector,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [6]:
#initialization from row vector(2D array)
vector = np.array([1.0,2.0,3.5])
vector_row = vector[np.newaxis,:]
neuron = Neuron(vector_row,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [7]:
#initialization from column vector(2D array)
vector = np.array([1.0,2.0,3.5])
vector_column = vector[:,np.newaxis]
neuron = Neuron(vector_column,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [8]:
#error messages testing
print("Test for: no function given as activation function")
try:
    vector = np.array([1.0,2.0,3.5])
    errorNeuron = Neuron(vector,1)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not vector given for weights initialization (2 dim)")
try:
    vector = np.array([[1.0,2.0,3.5],[1.0,2.0,3.5]])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not vector given for weights initialization (higher than 2 dim)")
try:
    vector = np.array([[[1.0,2.0,3.5],[1.0,2.0,3.5]],[[1.0,2.0,3.5],[1.0,2.0,3.5]]])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong weights given for initialization")
try:
    vector = np.array(["1.0","2.0","3.5"])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type given in list for weight initialization")
try:
    vector = ["as","2.0","3.5"]
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - inproper dimension of input")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = np.array([[[1.0,1.0],[1.0,1.0]],[[1.0,1.0],[1.0,1.0]]]) 
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - wrong value types in input: they are not rational numbers")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = np.array([1.0 + 3j,1.0]) 
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given variable is of not supported type for list")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = ["a","1.0"]
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given variable is of not supported type")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = "a"
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given input is too small to match neuron dimensions")
try:
    #wrong input (too small)
    vector = np.array([1.0,2.0,3.5])
    neuronTest = Neuron(vector,identity)
    inputTestErrorSmall = np.array([1.0])
    neuronTest.forward(inputTestErrorSmall)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given input is too small to match neuron dimensions")
try:
    #wrong input (too big)
    vector = np.array([1.0,2.0,3.5])
    neuronTest = Neuron(vector,identity)
    inputTestErrorBig = np.array([[1.0],[1.0],[1.0],[1.0]])
    neuronTest.forward(inputTestErrorBig)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Test for: no function given as activation function
Correct errors was caught. Error: Given input is not supported by implementation for activation function initialization. Given variable is not a function and thus can not be used as activation function.
Test for: not vector given for weights initialization (2 dim)
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given variable is not an vector (not column or row vector in 2 dim), so it is unsuitable for neuron weight initialization.
Test for: not vector given for weights initialization (higher than 2 dim)
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given variable is not an vector (due to more than 2 dim), so it is unsuitable for neuron weight initialization.
Test for: wrong weights given for initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization.

Layer class implementation testing

In [9]:
#initialization by number of dim in vector
dim_layer = np.array([4,5])
testLayer = Layer(dim_layer,identity,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [10]:
#initialization by number of dim in list
dim_layer = [4,5]
testLayer = Layer(dim_layer,identity,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [11]:
#initialization by giving weight array
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
testLayer = Layer(weights,identity,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [12]:
#initialization by giving weight array and function (single) from list
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
testLayer = Layer(weights,[identity],method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [13]:
#initialization by giving weight array and identity function (mutliple same) from list
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
activation_functions = [identity,identity,identity]
testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [14]:
#initialization by giving weight array and identity function (mutliple different) from list
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
activation_functions = [sigmoid,relu,leaky_relu]
testLayer = Layer(weights,activation_functions)
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [15]:
#error messages testing
print("Test for: wrong var type (not number) given in list for weight initialization")
try:
    vector = ["as","2.0"]
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type given (number, but no int) in list for weight initialization")
try:
    vector = [1,2.3]
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type (not number) given in 1D array for weight initialization")
try:
    vector = np.array(["as","2.0"])
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type given (number, but no int) in 1D array for weight initialization")
try:
    vector = np.array([1,2.3])
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong dimensions of given 1D array for weight initialization")
try:
    vector = np.array([1,2,1,1,2])
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not rational numbers given in 2D array for weight initialization")
try:
    weights = np.array([[1.0 + 4j,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not enough information given in 2D array for weight initialization")
try:
    weights = np.array([[]])
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: bigger than 2D was given for array for weight initialization")
try:
    weights = np.array([[[2]]])
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not supported data type given for array for weight initialization")
try:
    weights = "hehe"
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not supported data type given for array for weight initialization")
try:
    weights = "hehe"
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not compatible amount of activation functions given to the amount of neurons for activation function initialization")
try:
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,relu,leaky_relu,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not activation function given (inside list) for activation function initialization")
try:
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,1,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not activation function given (standalone) for activation function initialization")
try:
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = 1
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, too much dimensions in input")
try:
    #Input data for input propagation (forward pass)
    inputTestArray = np.array([[[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]],[[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]]])
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestArray)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, complex input")
try:
    #Input data for input propagation (forward pass)
    inputTestVect = np.array([[1.0 + 3j],[1.0],[1.0],[1.0],[1.0]])
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestVect)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, wrong data type of input")
try:
    #Input data for input propagation (forward pass)
    inputTestVect = "sda"
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestVect)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, wrong data type of input in list")
try:
    #Input data for input propagation (forward pass)
    inputTestList =["a",1.0,1.0,1.0,1.0]
    inputTestArray = np.array([[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]])
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestList)
    testLayer.forward(inputTestArray)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, input does not match layer neurons")
try:
    #Input data for input propagation (forward pass)
    inputTestList = [1.0,1.0,1.0,1.0,1.0,1.0,1.0]
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestList)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Test for: wrong var type (not number) given in list for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given values are not integers and thus can not represent of layer weights matrix.
Test for: wrong var type given (number, but no int) in list for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given values are not integers and thus can not represent of layer weights matrix.
Test for: wrong var type (not number) given in 1D array for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given values are not integers and thus can not represent of layer weights matrix.
Test for: wrong var type given (number, but no int) in 1D array for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weight

FNN class implementation testing

In [16]:
#initialization by number of dim in vector(list)
dim_layer = [5,5,3,2,1]
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
print(testNet.weights_list[0])
print(testNet.weights_list[1])
print(testNet.weights_list[2])
print(testNet.weights_list[3])
print(testNet.activ_functions_list_list[0])
print(testNet.activ_functions_list_list[1])
print(testNet.activ_functions_list_list[2])
print(testNet.activ_functions_list_list[3])
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

[[-0.94540119  0.59174498 -0.90446068  1.1309395   0.47138738 -0.88724937]
 [-0.28422792  0.68087239 -0.79866342  0.42600104 -0.73278385  0.24723089]
 [-1.84196573 -0.36452376  0.33590783  0.53783923  0.04846014  0.22981165]
 [-0.42347447 -0.22335574  0.05018507 -0.86345921  2.61258823  0.18173483]
 [-1.64935331  0.28258285 -0.2985092   1.46714406 -0.81210442  1.47692621]]
[[-3.68818642 -0.9630603  -0.71150739 -0.45152409 -0.23278334 -0.71737544]
 [-0.48562011  1.08768713  3.49573507  0.55887528  1.18727004  1.00625174]
 [-1.04664631  0.92795386 -2.08545491 -0.69083429  0.76867973 -0.92811869]]
[[ 1.79106471  0.28945822 -1.3654272  -0.57220429]
 [ 0.79637773  0.8840556  -0.02966232 -2.240352  ]]
[[-0.65151724  0.51916531 -1.29866682]]
[<function relu at 0x000001C13F7BF9C0>, <function relu at 0x000001C13F7BF9C0>, <function relu at 0x000001C13F7BF9C0>, <function relu at 0x000001C13F7BF9C0>, <function relu at 0x000001C13F7BF9C0>]
[<function relu at 0x000001C13F7BF9C0>, <function relu at 0

True

In [17]:
#initialization by number of dim in vector(1D array)
dim_layer = np.array([5,5,3,2,1])
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [18]:
#dummy network ini
dim_layer = np.array([5,5,3,2,1])
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#initialization by ready weight array
dim_layer = testNet.weights_list
print(dim_layer)
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

[array([[ 1.00603139, -2.18751562, -1.76918706, -0.56316092, -0.78573879,
         1.80295799],
       [ 0.60086099,  1.30060187,  0.5231211 , -0.08157263,  0.70640854,
        -1.26173633],
       [ 1.07182539, -0.7388798 ,  1.67523945, -0.13472887,  0.08978242,
        -0.41662265],
       [ 1.79659926, -0.33113309, -1.95269288,  2.65563121,  0.70151748,
         0.43209296],
       [-0.72165698,  0.4714956 , -1.51339407,  0.00882367, -0.07793316,
         1.47260871]]), array([[ 1.23585345, -0.05760911,  0.60138625, -0.01507612, -0.20920944,
         0.21583631],
       [ 1.24720824, -0.47055289, -0.90171235, -2.18722825,  0.30469817,
         0.56608477],
       [ 0.79590673, -0.8869606 , -0.27798922, -1.18380045,  1.36673503,
         0.58323239]]), array([[-0.90343118,  0.08752275, -0.25211843, -1.31368292],
       [ 0.7429236 ,  1.29502287,  0.55169808, -1.35535548]]), array([[ 0.77100093, -1.36764043, -0.45777526]])]


True

In [19]:
#initialization by number of dim in vector(1D array) and 1 activation function(as value)
dim_layer = np.array([5,5,3,2,1])
activ_func = identity
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [20]:
#initialization by number of dim in vector(1D array) and 1 activation function(as list)
dim_layer = np.array([5,5,3,2,1])
activ_func = [identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [21]:
#initialization by number of dim in vector(1D array) and 2 activation functions
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [22]:
#initialization by number of dim in vector(1D array) and activation function for each layer
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,sigmoid,leaky_relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [23]:
#initialization by number of dim in vector(1D array) and activation function for each neuron
dim_layer = np.array([5,5,3,2,1])
l1 = [relu,sigmoid,sigmoid,leaky_relu,identity]
l2 = [relu,sigmoid,sigmoid]
l3 = [relu,sigmoid]
l4 = [identity]
activ_func =  [l1,l2,l3,l4]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [24]:
#initialization by number of dim in vector(1D array) and 2 activation functions with Zero weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="Zero")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [25]:
#initialization by number of dim in vector(1D array) and 2 activation functions with random uniform weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomUni")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [26]:
#initialization by number of dim in vector(1D array) and 2 activation functions with random normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [27]:
#initialization by number of dim in vector(1D array) and 2 activation functions with Xavier uniform weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="XavUni")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [28]:
#initialization by number of dim in vector(1D array) and 2 activation functions with Xavier normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="XavNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [29]:
#initialization by number of dim in vector(1D array) and 2 activation functions with He uniform weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="HeUni")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [30]:
#initialization by number of dim in vector(1D array) and 2 activation functions with He normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="HeNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [31]:
#initialization by number of dim in vector(1D array) and 2 activation functions with He normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="HeNor")
#decomposing network
layers_list = testNet.decomposeIntoLayers()
neurons_list_list = testNet.decomposeIntoNeurons()
#checking the results
print(layers_list)
print(neurons_list_list)

[<Layer.Layer object at 0x000001C13F7E5B10>, <Layer.Layer object at 0x000001C13F7E52D0>, <Layer.Layer object at 0x000001C13F7E40D0>, <Layer.Layer object at 0x000001C13F7E4250>]
[[<Neuron.Neuron object at 0x000001C13F7E5890>, <Neuron.Neuron object at 0x000001C13F83BD10>, <Neuron.Neuron object at 0x000001C13F83A550>, <Neuron.Neuron object at 0x000001C13F8395D0>, <Neuron.Neuron object at 0x000001C13F839C90>], [<Neuron.Neuron object at 0x000001C13F838690>, <Neuron.Neuron object at 0x000001C13F83B4D0>, <Neuron.Neuron object at 0x000001C13F838FD0>], [<Neuron.Neuron object at 0x000001C13F83BB90>, <Neuron.Neuron object at 0x000001C13F839010>], [<Neuron.Neuron object at 0x000001C13F839B50>]]


In [32]:
#error messages testing
print("Test for: not supported input for weights")
try:
    dim_layer = "[5,5,3,2,1]"
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: array of not supported dimension")
try:
    dim_layer = np.array([[5,5,3,2,1],[5,5,3,2,1]])
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: given numbers are not integers")
try:
    dim_layer = np.array([5,5,3.1,2,1])
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not enough numbers given. Minimally required input and output layers cannot be created")
try:
    dim_layer = np.array([5])
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Test for: not supported input for weights
Correct errors was caught. Error:  Given input is not supported by implementation for weights initialization. Not supported data type given.
Test for: array of not supported dimension
Correct errors was caught. Error:  Given array is of size not supported by implementation. Supported dimensions: 1.
Test for: given numbers are not integers
Correct errors was caught. Error:  Given input is not supported by implementation for weights initialization. Given values in vector must be integers to properly represent number of neurons in layers
Test for: not enough numbers given. Minimally required input and output layers cannot be created
Correct errors was caught. Error:  Given input is not supported by implementation for weights initialization. At least two numbers are needed for FNN creation: FNN needs at least input and output layer (single layer network).


Reload Modules


In [33]:
import importlib
import sys
import numpy as np

# Reload order:
modules_in_order = [
    "ActivFunctions",
    "SuppFunctions",
    "InitFunctions",
    "Layer",
    "FNN",
    "TrainingFunctions"
]

for m in modules_in_order:
    if m in sys.modules:
        importlib.reload(sys.modules[m])
    else:
        globals()[m] = importlib.import_module(m)

print("All modules reloaded successfully.")


All modules reloaded successfully.


Imports

In [34]:
from FNN import FNN
from TrainingFunctions import backwards
from LossFunctions import MeanSquaredErrorDerivative

# Activations
from ActivFunctions import identity, sigmoid, tanh, relu, leaky_relu
import numpy as np


In [35]:
print("=== Test 1: Single-Layer Network Gradient ===")

dim_layer = np.array([4, 5])   # 4 inputs -> 5 outputs
testNet = FNN(dim_layer, identity, method_ini="RandomNor")

inputTestVect = np.ones((4, 1))
targetVect = 0.5 * np.ones((5, 1))

out = testNet.forward(inputTestVect)
grad = testNet.backward(out[0],out[1],targetVect,MeanSquaredErrorDerivative)

#grads = backwards(testNet, inputTestVect, targetVect, MeanSquaredErrorDerivative)

print("\nGradient matrix for single layer:")
print(grad[0])
print(grad)


=== Test 1: Single-Layer Network Gradient ===

Gradient matrix for single layer:
[[ 2.2944065   2.2944065   2.2944065   2.2944065   2.2944065 ]
 [ 0.95506836  0.95506836  0.95506836  0.95506836  0.95506836]
 [ 0.88528492  0.88528492  0.88528492  0.88528492  0.88528492]
 [-0.34688129 -0.34688129 -0.34688129 -0.34688129 -0.34688129]
 [-1.17380486 -1.17380486 -1.17380486 -1.17380486 -1.17380486]]
[array([[ 2.2944065 ,  2.2944065 ,  2.2944065 ,  2.2944065 ,  2.2944065 ],
       [ 0.95506836,  0.95506836,  0.95506836,  0.95506836,  0.95506836],
       [ 0.88528492,  0.88528492,  0.88528492,  0.88528492,  0.88528492],
       [-0.34688129, -0.34688129, -0.34688129, -0.34688129, -0.34688129],
       [-1.17380486, -1.17380486, -1.17380486, -1.17380486, -1.17380486]])]


In [36]:
print("=== Test 1: Single-Layer Network Gradient with multi column input ===")

dim_layer = np.array([4, 5])   # 4 inputs -> 5 outputs
testNet = FNN(dim_layer, softmax, method_ini="RandomNor")

inputTestVectt = np.ones((4, 1))
inputTestArray = np.concatenate((inputTestVectt,inputTestVectt), axis = 1) 
targetVectt = 0.5 * np.ones((5, 1))
targetTestArray = np.concatenate((targetVectt,targetVectt), axis = 1) 
out = testNet.forward(inputTestArray)
grad = testNet.backward(out[0],out[1],targetTestArray,MeanSquaredErrorDerivative)

#grads = backwards(testNet, inputTestVect, targetVect, MeanSquaredErrorDerivative)

print("\nGradient matrix for single layer:")
print(grad[0])
print(grad)

=== Test 1: Single-Layer Network Gradient with multi column input ===

Gradient matrix for single layer:
[[-0.09436923 -0.09436923 -0.09436923 -0.09436923 -0.09436923]
 [-0.0613928  -0.0613928  -0.0613928  -0.0613928  -0.0613928 ]
 [-0.07723467 -0.07723467 -0.07723467 -0.07723467 -0.07723467]
 [-0.03269326 -0.03269326 -0.03269326 -0.03269326 -0.03269326]
 [ 0.01361091  0.01361091  0.01361091  0.01361091  0.01361091]]
[array([[-0.09436923, -0.09436923, -0.09436923, -0.09436923, -0.09436923],
       [-0.0613928 , -0.0613928 , -0.0613928 , -0.0613928 , -0.0613928 ],
       [-0.07723467, -0.07723467, -0.07723467, -0.07723467, -0.07723467],
       [-0.03269326, -0.03269326, -0.03269326, -0.03269326, -0.03269326],
       [ 0.01361091,  0.01361091,  0.01361091,  0.01361091,  0.01361091]])]


In [59]:
print("=== Test 2: Deep 10-Layer Network ===")

dim_layer = np.array([
    5, 8, 6, 7, 5, 9, 4, 3, 2, 4, 1
])

deepNet = FNN(dim_layer, [relu,softmax], method_ini="RandomNor")

inputVect = np.random.normal(1,10,(5, 1))
targetVect = np.array([[0.7]])

out = deepNet.forward(inputVect)
grad = deepNet.backward(out[0],out[1],targetVect,MeanSquaredErrorDerivative)

#grads = backwards(deepNet, inputVect, targetVect, MeanSquaredErrorDerivative)

for i, g in enumerate(grad):
    print(f"\nGradient matrix for layer {i}:")
    print(g)


=== Test 2: Deep 10-Layer Network ===

Gradient matrix for layer 0:
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]

Gradient matrix for layer 1:
[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]

Gradient matrix for layer 2:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]

Gradient matrix for layer 3:
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]

Gradient matrix for layer 4:
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0.

In [56]:
print("=== Test 2: Deep 10-Layer Network with multi column input ===")

dim_layer = np.array([
    5, 8, 6, 7, 5, 9, 4, 3, 2, 4, 1
])

deepNet = FNN(dim_layer, [relu,softmax], method_ini="RandomNor")

inputTestVectt =  np.ones((5, 1))
inputTestArray = np.concatenate((inputTestVectt,inputTestVectt), axis = 1) 
targetVectt = np.array([[0.7]])
targetTestArray = np.concatenate((targetVectt,targetVectt), axis = 1) 

#inputVect = np.ones((5, 1))
#targetVect = np.array([[0.7]])

out = deepNet.forward(inputTestArray)
grad = deepNet.backward(out[0],out[1],targetTestArray,MeanSquaredErrorDerivative)

#grads = backwards(deepNet, inputVect, targetVect, MeanSquaredErrorDerivative)

for i, g in enumerate(grad):
    print(f"\nGradient matrix for layer {i}:")
    print(g)

=== Test 2: Deep 10-Layer Network with multi column input ===


KeyboardInterrupt: 